Mathematical trick behing self attention

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [111]:
torch.manual_seed(1337)
B, T, C= 2, 4, 3       # batch, time, channels
x = torch.randn(B, T, C)
x.shape


torch.Size([2, 4, 3])

Tensor Slicing Based on Dimensions
For a tensor x of arbitrary shape (D1, D2, D3, ...):

Select All Elements:<br/>
x[:]: Selects the entire tensor. <br/>
x[:, :, :]: Selects all elements across all dimensions. <br/>

Single Dimension Slicing:<br/>
x[start:stop]: Slices only along the first dimension.<br/>
x[:, start:stop]: Slices along the second dimension.<br/>
x[:, :, start:stop]: Slices along the third dimension.<br/>

Specific Index Selection:<br/>
x[index]: Selects a specific index along the first dimension, reducing dimensionality.<br/>
x[:, index]: Selects a specific index along the second dimension.<br/>
x[:, :, index]: Selects a specific index along the third dimension.<br/>

Stepwise Slicing:<br/>
x[::2]: Selects every second element along the first dimension.<br/>
x[:, ::2]: Selects every second element along the second dimension.<br/>
x[:, :, ::2]: Selects every second element along the third dimension.<br/>

Reverse Slicing:<br/>
x[::-1]: Reverses the first dimension.<br/>
x[:, ::-1]: Reverses the second dimension.<br/>
x[:, :, ::-1]: Reverses the third dimension.<br/>

In [112]:
#we want x[b, t] = mean_{i<=t} x[b, i]
def batch_mean(B, T, C):
    x = torch.randn(B, T, C)
    xbatch_mean = torch.zeros((B, T, C))
    for b in range(B):                          #this means for each batch
        for t in range(T):                      #for every time frame or row
            xprev= x[b, :t+1] #(t, C)           #slicing along time dimension, so the shape becomes of (t, C) as batch is constant due to for each batch
            xbatch_mean[b, t] = torch.mean(xprev, axis= 0)   #mean along rows due to axis= 0
    return x, xbatch_mean

In [113]:
x, xbatch_mean = batch_mean(B=2, T=3, C=4)

In [114]:
x

tensor([[[ 1.1490,  0.1812, -0.0920,  1.5828],
         [ 0.1526,  0.3843,  1.3091,  0.4645],
         [-0.8345,  0.5978, -0.0514, -0.0646]],

        [[-0.4970,  0.4658, -0.2573, -1.0673],
         [ 2.0089, -0.5370,  0.2228,  0.6971],
         [-1.4267,  0.9059,  0.1446,  0.2280]]])

In [115]:
xbatch_mean

tensor([[[ 1.1490,  0.1812, -0.0920,  1.5828],
         [ 0.6508,  0.2827,  0.6085,  1.0237],
         [ 0.1557,  0.3878,  0.3886,  0.6609]],

        [[-0.4970,  0.4658, -0.2573, -1.0673],
         [ 0.7560, -0.0356, -0.0172, -0.1851],
         [ 0.0284,  0.2782,  0.0367, -0.0474]]])

from above we can see for every time step it is averaging its past time in channel dimension, in every batch 
like in batch 1 we can see first row remains same
2 row becomes average of first and second row. and this row is the time.
so on till last row

and again same in batch 2

# Matrix Multiplication

In [116]:
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
def MatMul(a, b):
    c = a @ b 
    print("a=")
    print(a)
    print("b=")
    print(b)
    return c

In [117]:
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
print(MatMul(a,b))

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[6., 9.],
        [9., 2.],
        [8., 9.]])
tensor([[23., 20.],
        [23., 20.],
        [23., 20.]])


For lower triangular matrix

In [118]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [119]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim= 1, keepdim= True)
print(a)
b = torch.randint(0,10, (3,3)).float()
x_matmul_mean= MatMul(a, b)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[1., 6., 1.],
        [9., 0., 1.],
        [9., 8., 4.]])


Here we can conclude that the xbatch_mean and c are basically resulting similar, 
ie keeping the first time or row constant
and from second row it is doing average

In [120]:
x[0], xbatch_mean[0]

(tensor([[ 1.1490,  0.1812, -0.0920,  1.5828],
         [ 0.1526,  0.3843,  1.3091,  0.4645],
         [-0.8345,  0.5978, -0.0514, -0.0646]]),
 tensor([[ 1.1490,  0.1812, -0.0920,  1.5828],
         [ 0.6508,  0.2827,  0.6085,  1.0237],
         [ 0.1557,  0.3878,  0.3886,  0.6609]]))

In [121]:
b, x_matmul_mean

(tensor([[1., 6., 1.],
         [9., 0., 1.],
         [9., 8., 4.]]),
 tensor([[1.0000, 6.0000, 1.0000],
         [5.0000, 3.0000, 1.0000],
         [6.3333, 4.6667, 2.0000]]))

both xbatch_mean and x_matmul_mean is giving result with same logic

In [122]:
torch.manual_seed(1337)
B, T, C= 4, 8, 2       # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [123]:

wei= torch.tril(torch.ones(T, T))
wei= wei / wei.sum(dim= 1, keepdim= True)
x_batch_mean= wei @ x

In [124]:
x[0], x_batch_mean[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In Transformer researchpaper the attention is calculated using softmax, so lets try softmax

In [125]:
import torch.nn.functional as F

In [126]:
#Softmax
tril= torch.tril(torch.ones(T, T))
wei= torch.zeros((T, T))
wei= wei.masked_fill(tril == 0, float('-inf'))
wei= F.softmax(wei, dim= -1)
x_batch_mean_softmax = wei @ x

In [127]:
x[0], x_batch_mean_softmax[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

This is also showing same result, as we wanted

In [128]:
import torch.nn as nn

In [135]:
#self attention
"""
the wei(weight) or tokens couldnot be uniform, above placed zeros.
because different tokens finds different other tokens more or less interesting.
and we want this to be data dependent, examples; vowels looks for the consonent in the past
so now i want to gather the information from the past but i wanted to be in data dependent way.
this is the problem that self attention solves.
This is how self attention solves:
for every single node or token at each position will two vector.
query vector; what am i looking for
key vector; what do i get
so to get the affinities or similiraties between them, we dot product them.
now that dot product between query and key gives the similarity between tokens.
and here wei is that dot product.
"""

head_size = 16
key = nn.Linear(C, head_size, bias= False)
query = nn.Linear(C, head_size, bias= False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      #(B, T, 16)
q = query(x)    #(B, T, 16)
wei = q @ k.transpose(-2,-1)  #(B,T,16) @ (B,16,T)--> (B,T,T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v= value(x)
out = wei @ v
"""
we can think like x is like the private token. 
and value is if you find me relevant (based on the key), this is the information I’ll give you.
"""
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

if we remove the masking of wei then it becomes the encoder, cause 
in decoder block there is autoregressive format where it predicts one token at a time based on previous context
but in encoder we allow all the nodes or token to completely talk to each other. 

this is self attention because key query and value are from same input x, cross attention means k and v from external source but query produced from input x

In [136]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5572, 0.4428, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2462, 0.5681, 0.1857, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1756, 0.6245, 0.1141, 0.0858, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2107, 0.2477, 0.1696, 0.1438, 0.2283, 0.0000, 0.0000, 0.0000],
        [0.0845, 0.0158, 0.1586, 0.2427, 0.4899, 0.0084, 0.0000, 0.0000],
        [0.1360, 0.1998, 0.1237, 0.1168, 0.0839, 0.1961, 0.1437, 0.0000],
        [0.0401, 0.0075, 0.0564, 0.0686, 0.3650, 0.0103, 0.0306, 0.4215]],
       grad_fn=<SelectBackward0>)

looking at tensor we can conclude, in one time frame the high value between tokens tells us that those token are of high affinity or finds each other more interesting

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [145]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei= q @ k.transpose(-2, -1) * head_size**-0.5

In [146]:
k.var()

tensor(1.0632)

In [147]:
q.var()

tensor(1.0564)

In [148]:
wei.var()

tensor(1.1176)

the reason of dividing it with sqrt of head_size is that, in weight initialization if the values are huge positive or huge negative then softmax vector will converge towards one hot vectors, which doesnot show data dependent

In [156]:
#if the values are like this then look the softmax became too much peaky, for token 0.5
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
#here we find token 0.3 and 0.5 are nearly same during softmax, so we want something like this 
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

to get the softmax values like this, we make the wei variance nearly equal to 1, by dividing it with sqrt of headsize

# Multihead attention 
meaning multiple head of self attention running in parallel.
run multiple head in parallel into a list and simply concatinate it over the channel dimension.

since we have n_embed of size 32, if we have the the 4 channels then we have 8 dimensional self attention
so from each communication channel we will have 8 dimensional vectors and those 4 channel concat to form the original n_embed size

# FeedForward
self attention is the communication, and once they gathered all the data they need to think on their data individually.
and thats what feed forward does

Block does communication followed by computation 
communication is done by multi-head attention
computation is done by feed forward network on each token independently

There are two optimization that help with the depth of this network
Residual connections and layernorm

# Residual connection
Residual connections in transformers ensure efficient gradient flow, prevent vanishing/exploding gradients, and allow the network to learn incrementally. This results in faster convergence, better optimization, and the ability to train deeper networks effectively. They help maintain stability and improve performance in the complex transformer architecture, especially as the model depth increases.

In the self-attention mechanism, the process involves computing queries (Q), keys (K), and values (V), followed by an attention operation and a linear transformation. This results in a series of complex operations that can be difficult to optimize without residual connections.

Skip Pathways for Attention Outputs: Each multi-head attention layer produces its own set of attention outputs, but instead of letting the output go through multiple transformations without control, a residual connection ensures that the input (the original query) is added back to the final output.

The equation in the attention mechanism is typically:

output=LayerNorm(QK^T +V)+input

The output of the self-attention layer is added to the input to form the residual connection. This means that even if the attention mechanism fails to capture all useful information (e.g., poor attention weights), the original input 
x still contributes directly to the final output.

This enables the model to retain useful information from previous layers and learn more efficiently because it has a stable reference to the input in every layer.

# Layer Normalization:
Batch normalization means across the batch dimension any individual neuron had unit Gaussian distribution. mean=0 and standard deviation=1
and in layernorm normalizes the features, it normalizes row instead of column, 

Transformers typically use Layer Normalization after the attention and feed-forward layers. This helps improve optimization by stabilizing the training process. The residual connections, when combined with Layer Normalization, make sure the distribution of activations is stable and prevents the network from diverging.

The equation for Layer Normalization with residual connections is:

output=LayerNorm(input+F(input))

Where 
F(input) represents the transformation (e.g., self-attention or feed-forward network).

Improved Training Stability: The residual connection in each Transformer layer ensures that the model can learn incrementally rather than having to learn large transformations all at once. This incremental learning approach makes the optimization process more stable.


# Dropout
Randomly preventing neighbout neurons to communicate by making it off

# Residual Connections in Deep Neural Networks

When you have residual connections in a deep neural network, the layers are learning residuals, or differences between the input and output, rather than the entire transformation from input to output. This creates a shortcut path for the data to flow through the network.

This shortcut is essentially a way for the network to learn identity mappings if necessary. Let me explain how that works:

## 1. Deep Network Without Residuals (Problematic Case):
In a very deep network without residual connections, the deeper layers need to learn the complete transformation \( H(x) \).
As we add more layers, the learning process becomes harder, and the gradients may vanish or explode due to the increasing depth.
This can cause the network to fail to learn effectively in deeper layers, and the performance could degrade as we add more layers. Essentially, the deep network might perform worse than a shallower network because each layer is having trouble learning useful features.

## 2. Deep Network With Residuals (How it Helps):
With residual connections, each layer doesn't need to learn the entire transformation from input to output. Instead, it only needs to learn the residual, i.e., the difference between the input and output:

\[
y = F(x) + x
\]

Where \( F(x) \) is the learned residual, and \( x \) is the input passed directly through.

Now, if the network cannot learn anything useful in the deeper layers (e.g., if the gradient vanishes or the weights don’t adjust properly), the residual \( F(x) \) can just become zero:

\[
F(x) = 0
\]

Which leads to:

\[
y = x
\]

This means that the deeper layers would just output the input unchanged, essentially learning the identity function.

## 3. What Does This Do for the Network's Performance?

- **No Degradation**: When the deeper layers learn the identity function (i.e., \( F(x) = 0 \)), the output is the same as the input \( y = x \). This is equivalent to having a shallow network. In other words, the network would not perform worse than a shallow network.

- **Avoiding Degradation**: The key point is that the residual network will not perform worse than its shallower counterpart. If the deeper layers fail to learn, they can simply pass the input through as it is, thus ensuring that the deeper network performs at least as well as the shallower one.

- **Improvement in Training**: This flexibility allows the deeper network to continue training without the risk of degradation. The network can still learn useful features if the residual part \( F(x) \) is nonzero, but if not, it falls back to the identity function, which is still a valid solution. This helps avoid the network performance deteriorating as layers are added, which is the degradation problem.

## Summary:
- In a residual network, the deeper layers can always choose to learn the identity function (i.e., they can pass the input through unchanged).
- This means that, even if the deeper layers cannot learn anything useful, the network will not perform worse than a shallow network.
- The flexibility to learn the identity function prevents the network from suffering from performance degradation when more layers are added.
- The network ensures at least the same performance as a shallow network, or even better if the deeper layers can learn useful transformations.


# Backpropagation Explained with Example

### **1. Forward Pass**
In the forward pass, we compute the output (`y_hat`) and the loss (`L`) based on the current weights and biases. The goal of this step is to obtain the predicted output so that we can calculate the error.

- **What Happens:**
  - Compute the linear combinations (`z[l] = W[l] * a[l-1] + b[l]`).
  - Apply the activation function (`a[l] = f(z[l])`).
  - Continue this process layer by layer until you reach the final output (`y_hat`).
  - Calculate the loss (`L`) to measure the error between `y_hat` and the true label (`y`).

- **Example Insight:**
  If the network's prediction (`y_hat`) is far from the true value (`y`), the loss `L` will be large, indicating that significant updates to the weights are required.

---

### **2. Loss Gradient at the Output Layer**
The first step in backpropagation is to calculate how the loss changes with respect to the output (`y_hat`).

- **What Happens:**
  - Compute the gradient of the loss with respect to `y_hat` (`∂L/∂y_hat`).
  - This tells us how much the output is contributing to the error.

- **Example Insight:**
  If `L = 1/2 * (y_hat - y)^2`, then `∂L/∂y_hat = y_hat - y`.  
  For `y_hat = 0.8` and `y = 1.0`, `∂L/∂y_hat = 0.8 - 1.0 = -0.2`.

---

### **3. Compute Error at the Output Layer**
Next, the error at the output layer (`delta[L]`) is computed by incorporating the derivative of the activation function at the output.

- **What Happens:**
  - `delta[L] = ∂L/∂y_hat * f'(z[L])`, where `f'(z[L])` is the derivative of the activation function used in the output layer.
  - This error term quantifies how much the output neuron needs to adjust its parameters to reduce the loss.

- **Example Insight:**
  For sigmoid activation:  
  `f'(z) = f(z) * (1 - f(z))`.  
  If `y_hat = 0.8`, then `f'(z[L]) = 0.8 * (1 - 0.8) = 0.16`.  
  `delta[L] = -0.2 * 0.16 = -0.032`.

---

### **4. Compute Gradients for Weights and Biases at the Output Layer**
Using the error `delta[L]`, compute how the weights and biases in the output layer affect the loss.

- **What Happens:**
  - Gradients for weights: `∂L/∂W[L] = delta[L] * a[L-1]`, where `a[L-1]` is the activation from the previous layer.
  - Gradients for biases: `∂L/∂b[L] = delta[L]`.

- **Example Insight:**
  If `delta[L] = -0.032` and `a[L-1] = 0.75`, then:  
  `∂L/∂W[L] = -0.032 * 0.75 = -0.024`,  
  `∂L/∂b[L] = -0.032`.

---

### **5. Backpropagate Error to Hidden Layers**
The error is propagated backward through the network. For hidden layers, we calculate the error using the chain rule.

- **What Happens:**
  - Error at layer `l`:  
    `delta[l] = W[l+1]T * delta[l+1] * f'(z[l])`.
  - This combines the contributions from the errors in the subsequent layer (`delta[l+1]`) and the derivative of the activation at the current layer.

- **Example Insight:**
  If `W[2] = 0.6`, `delta[2] = -0.032`, and `f'(z[1]) = 0.18`,  
  `delta[1] = 0.6 * -0.032 * 0.18 = -0.003456`.

---

### **6. Compute Gradients for Weights and Biases in Hidden Layers**
Using the errors in the hidden layers, compute the gradients for the weights and biases.

- **What Happens:**
  - Gradients for weights: `∂L/∂W[l] = delta[l] * a[l-1]`.
  - Gradients for biases: `∂L/∂b[l] = delta[l]`.

- **Example Insight:**
  If `delta[1] = -0.003456` and `a[0] = x = 2.0`, then:  
  `∂L/∂W[1] = -0.003456 * 2.0 = -0.006912`,  
  `∂L/∂b[1] = -0.003456`.

---

### **7. Update Weights and Biases**
Using the computed gradients, update the parameters using gradient descent.

- **What Happens:**
  - Update weights:  
    `W[l] = W[l] - η * ∂L/∂W[l]`.
  - Update biases:  
    `b[l] = b[l] - η * ∂L/∂b[l]`.

- **Example Insight:**
  For `W[1] = 0.5`, `∂L/∂W[1] = -0.006912`, and `η = 0.1`:  
  `W[1] = 0.5 - 0.1 * -0.006912 = 0.5006912`.

---

### **8. Repeat for Multiple Iterations**
The process of forward pass, backward pass, and parameter updates is repeated for multiple epochs until the loss is minimized and the network predictions (`y_hat`) align closely with the true labels (`y`).


### Example Walkthrough (Simple Case)

Consider the following:
- Input: x = 2
- Target output: y_true = 3
- Initial weight: W = 0.5
- Learning rate: eta = 0.1

#### **Iteration 1:**

1. **Forward Pass:**
   - Predicted output: 
     y_pred = W * x = 0.5 * 2 = 1
   - Loss:
     Loss = 1/2 * (y_pred - y_true)^2 = 1/2 * (1 - 3)^2 = 2

2. **Backward Pass (Gradient Computation):**
   - Gradient of the loss with respect to W:
     dL/dW = (y_pred - y_true) * 2 = (1 - 3) * 2 = -4

3. **Weight Update:**
   - Update weight W:
     W = W - eta * dL/dW = 0.5 - 0.1 * (-4) = 0.5 + 0.4 = 0.9

#### **Iteration 2:**

1. **Forward Pass with Updated Weight:**
   - Predicted output:
     y_pred = W * x = 0.9 * 2 = 1.8
   - Loss:
     Loss = 1/2 * (y_pred - y_true)^2 = 1/2 * (1.8 - 3)^2 = 0.72

2. **Backward Pass (Gradient Computation):**
   - Gradient of the loss with respect to W:
     dL/dW = (y_pred - y_true) * 2 = (1.8 - 3) * 2 = -2.4

3. **Weight Update:**
   - Update weight W:
     W = 0.9 - 0.1 * (-2.4) = 0.9 + 0.24 = 1.14

---

This process repeats, with the weight getting closer to a value that minimizes the loss, ultimately making the model's predictions closer to the true labels.
